In [1]:
import pandas as pd
import json
import ast
import os
from tqdm import tqdm
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD

In [4]:
data = pd.read_csv(r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\dataframes\playlists_trackId.csv')

In [6]:
# Parse the tracks column into actual lists of integers
data["tracks"] = data["tracks"].apply(lambda x: list(map(int, ast.literal_eval(x))))

# Display the transformed DataFrame
print(data)

        user_id                                             tracks
0             0  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
1             1  [52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 6...
2             2  [91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101,...
3             3  [155, 156, 157, 158, 159, 160, 161, 162, 163, ...
4             4  [281, 282, 283, 284, 285, 286, 287, 288, 289, ...
...         ...                                                ...
999995   999995  [116671, 5783, 48788, 5784, 91283, 17696, 5046...
999996   999996  [2262285, 2262286, 2262287, 2262288, 2262289, ...
999997   999997  [39643, 41143, 5434, 22056, 3860, 1043, 9152, ...
999998   999998  [5768, 5872, 14504, 3936, 76224, 5838, 5838, 2...
999999   999999  [12421, 12518, 14294, 268821, 311299, 92033, 1...

[1000000 rows x 2 columns]


In [7]:
# Step 1: Extract rows, columns, and values for the sparse matrix
rows = []
cols = []
values = []

for index, row in tqdm(data.iterrows()):
    try:
        user_id = row["user_id"]  # Extract user_id
        tracks = row["tracks"]    # Extract tracks
        rows.extend([user_id] * len(tracks))  # Repeat user_id for each track
        cols.extend(tracks)  # Track indices as columns
        values.extend([1] * len(tracks))  # All values are 1
    except Exception as e:
        print(f'Error at index {index}: {e}')

1000000it [00:51, 19399.78it/s]


In [8]:
# Save the DataFrame to a file
salva = False
if salva:
    output_dir = r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\dataframes'
    os.makedirs(output_dir, exist_ok=True)

    # Save as CSV instead of Parquet
    output_file = os.path.join(output_dir, "playlists_trackId.csv")
    data.to_csv(output_file, index=False)


    print(f"Processing completed. DataFrame saved to {output_file}.")

In [9]:
# Step 2: Create the sparse matrix
sparse_matrix = coo_matrix((values, (rows, cols)), shape=(1000001, 2262293))

In [37]:
print(sparse_matrix)

<COOrdinate sparse matrix of dtype 'int64'
	with 66346428 stored elements and shape (1000001, 2262293)>
  Coords	Values
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1
  (0, 21)	1
  (0, 22)	1
  (0, 23)	1
  (0, 24)	1
  (0, 25)	1
  :	:
  (999999, 3897)	1
  (999999, 83428)	1
  (999999, 340556)	1
  (999999, 194410)	1
  (999999, 41163)	1
  (999999, 12489)	1
  (999999, 263925)	1
  (999999, 101245)	1
  (999999, 327267)	1
  (999999, 250248)	1
  (999999, 250249)	1
  (999999, 128315)	1
  (999999, 221578)	1
  (999999, 12426)	1
  (999999, 41167)	1
  (999999, 153549)	1
  (999999, 466735)	1
  (999999, 466733)	1
  (999999, 72395)	1
  (999999, 72411)	1
  (999999, 72433)	1
  (999999, 288295)	1
  (999999, 141055)	1
  (999999, 141040)	1
  (999999, 72396)	1


In [10]:
# Step 3: Apply Truncated SVD
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD

# Convert COO to CSR for better compatibility
csr = sparse_matrix.tocsr()
csr = csr.astype(float)

from scipy.sparse.linalg import svds

# Perform Truncated SVD with a limited number of components
k = 1e6  # Number of components
U, S, Vt = svds(csr, k=k)

print("U Matrix:", U)
print("Singular Values:", S)
print("Vt Matrix:", Vt)


MemoryError: Unable to allocate 7.28 TiB for an array with shape (1000001, 1000001) and data type float64

In [7]:
# Create a matrix where rows are users and columns are movies, and the values are ratings
utility_matrix = data.pivot(index='user_id', columns='movieId', values='rating').fillna(0)
print("Utility Matrix Shape:", utility_matrix.shape)

# Convert to numpy array
R = utility_matrix.to_numpy()

KeyError: 'userId'